# Librerias y dependencias

In [4]:
# Manejo de direcotrios
import os
# Manejo de archivos
import imageio
import tempfile

# Manejo de fechas
from datetime import datetime

# Manejo y procesamiento de datos
import pandas as pd
import numpy as np

# Graficos de resultados
import math
import matplotlib.pyplot as plt

# Modelo de prediccion
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_absolute_percentage_error

# Generacion de mapas
import folium
from folium.plugins import HeatMap, HeatMapWithTime, MarkerCluster

C:\Users\Estudiante\Proyecto-bootcamp\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


# Entrenamiento

## Preparativos

### Parametros

In [26]:
DATOS_PATH = r"Datasets\Datos_de_entrenamiento.csv"
DIR_PATH = r"Datasets\Directorio_municipios_procesado.csv"
OUTPUT_DIR = r"Resultados prediccion Top 10"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Fecha limite de prediccion
PRED_END = "2025-12-31"

# Selección: si deseas entrenar todos los municipios pon None, si prefieres top_k por visitas totales pon un entero
TOP_K = 10
MIN_MONTHS_FOR_MODEL = 6 # mínimo meses de datos para entrenar

### Funciones auxiliares

In [27]:
def prepare_prophet_df(df, municipio):
    d = df[df['MUNICIPIO'] == municipio][['ds','VISITAS']].rename(columns={'VISITAS':'y'}).sort_values('ds')
    # llenar meses faltantes para continuidad (freq = MS)
    if d.empty:
        return d
    idx = pd.date_range(d['ds'].min(), d['ds'].max(), freq='MS')
    d = d.set_index('ds').reindex(idx).rename_axis('ds').reset_index()
    d['y'] = d['y'].fillna(0)
    return d

def train_and_forecast(df_ts, periods_months):
    m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
    # añadir seasonality mensual si se quiere:
    m.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    m.fit(df_ts)
    future = m.make_future_dataframe(periods=periods_months, freq='MS')
    forecast = m.predict(future)
    return m, forecast

def months_between_dates(start, end):
    s = pd.to_datetime(start)
    e = pd.to_datetime(end)
    return (e.year - s.year) * 12 + (e.month - s.month)

## Carga y procesamiento

In [28]:
# Carga y preprocesamiento básico
df_raw = pd.read_csv(DATOS_PATH)
df_dir = pd.read_csv(DIR_PATH)


# Esperamos: df_raw con MES,AÑO,MUNICIPIO,VISITAS
# Crear columna fecha (usaremos primer día del mes)
df_raw['MES'] = df_raw['MES'].astype(int)
df_raw['AÑO'] = df_raw['AÑO'].astype(int)
df_raw['ds'] = pd.to_datetime(df_raw['AÑO'].astype(str) + '-' + df_raw['MES'].astype(str).str.zfill(2) + '-01')


# Merge para unir coordenadas (puede haber municipios sin coordenadas)
df_merge = df_raw.merge(df_dir[['MUNICIPIO','LONGITUD','LATITUD']], on='MUNICIPIO', how='left')


agg_total = df_merge.groupby('MUNICIPIO', as_index=False)['VISITAS'].sum().sort_values('VISITAS', ascending=False)
if TOP_K is None:
    lstMunicipios = agg_total['MUNICIPIO'].tolist()
else:
    lstMunicipios = agg_total.head(TOP_K)['MUNICIPIO'].tolist()

print(f"Cantidad de municipios seleccionados: {len(lstMunicipios)}")
print(f"Municipios seleccionados: {lstMunicipios}")


# Preparar horizonte en meses desde último dato hasta PRED_END
last_date = df_merge['ds'].max()
print('Último dato en:', last_date)
periods_to_predict = months_between_dates(last_date, PRED_END)
print('Meses a predecir:', periods_to_predict)

Cantidad de municipios seleccionados: 10
Municipios seleccionados: ['PAIPA', 'VILLA DE LEYVA', 'DUITAMA', 'AQUITANIA', 'TUNJA', 'IZA', 'MONGUI', 'TOTA', 'NOBSA', 'SOGAMOSO']
Último dato en: 2024-10-01 00:00:00
Meses a predecir: 14


## Entrenamiento y Evaluacion

### Entrenamiento

In [29]:
results = {}
for municipio in lstMunicipios:
    df_ts = prepare_prophet_df(df_merge, municipio)
    if df_ts.shape[0] < MIN_MONTHS_FOR_MODEL:
        print(f"Omitido {municipio}: solo {df_ts.shape[0]} meses de datos")
        continue
    try:
        model, forecast = train_and_forecast(df_ts, periods_to_predict)
        results[municipio] = {'model': model, 'forecast': forecast, 'history': df_ts}
        # Guardar gráfica simple
        fig = model.plot(forecast)
        path = os.path.join(OUTPUT_DIR, municipio)
        os.makedirs(path, exist_ok=True)
        fig.savefig(os.path.join(path, f'forecast.png'), bbox_inches='tight')
        plt.close(fig)
        comp = model.plot_components(forecast)
        comp.savefig(os.path.join(OUTPUT_DIR, municipio, f'components.png'), bbox_inches='tight')
        plt.close(comp)
        print(f"Modelo generado para {municipio}")
    except Exception as e:
        print('Error en ', municipio, e)

19:01:09 - cmdstanpy - INFO - Chain [1] start processing
19:01:12 - cmdstanpy - INFO - Chain [1] done processing
19:01:13 - cmdstanpy - INFO - Chain [1] start processing


Modelo generado para PAIPA


19:01:33 - cmdstanpy - INFO - Chain [1] done processing
19:01:34 - cmdstanpy - INFO - Chain [1] start processing


Modelo generado para VILLA DE LEYVA


19:01:48 - cmdstanpy - INFO - Chain [1] done processing
19:01:48 - cmdstanpy - INFO - Chain [1] start processing


...

Modelo generado para TOTA


19:03:00 - cmdstanpy - INFO - Chain [1] done processing
19:03:01 - cmdstanpy - INFO - Chain [1] start processing


Modelo generado para NOBSA


19:03:17 - cmdstanpy - INFO - Chain [1] done processing


Modelo generado para SOGAMOSO


### Evaluacion

In [30]:
# Evaluación MAPE(Mean Absolute Percentage Error) que significa Error Porcentual Absoluto Medio.
#  sobre el período observado usando los puntos de backtest en forecast

evals = []
for muni, obj in results.items():
    hist = obj['history']
    f = obj['forecast']
    # tomar solo las fechas que estaban en el histórico
    f_obs = f[f['ds'].isin(hist['ds'])]
    if f_obs.empty:
        continue
    y_true = hist['y'].values
    y_pred = f_obs['yhat'].values
    mape = mean_absolute_percentage_error(y_true, y_pred)
    evals.append({'MUNICIPIO': muni, 'MAPE': mape})


pd.DataFrame(evals).sort_values('MAPE').to_csv(os.path.join(OUTPUT_DIR, 'evaluation_mape.csv'), index=False)

# Visualizacion de resultados


## Preparacion datos mensuales por municipio

## Estandarizacion de datos

In [32]:
# Preparar data mensual por municipio para el mapa (unir histórico + predicción)
# Vamos a construir una tabla con columnas: ds, MUNICIPIO, VISITAS (observado o predicho), LONGITUD, LATITUD
map_rows = []
for muni, obj in results.items():
    f = obj['forecast'][['ds','yhat']].copy().rename(columns={'yhat':'VISITAS'})
    last_hist = obj['history']['ds'].max()
    f['TIPO'] = np.where(f['ds'] <= last_hist, 'OBS', 'PRED')
    # adjuntar coordenadas
    coords_df = df_dir[df_dir['MUNICIPIO']==muni][['LONGITUD','LATITUD']]
    if not coords_df.empty:
        coords = coords_df.iloc[0].to_dict()
        f['LONGITUD'] = coords.get('LONGITUD', None)
        f['LATITUD'] = coords.get('LATITUD', None)
    else:
        f['LONGITUD'] = np.nan
        f['LATITUD'] = np.nan
    f['MUNICIPIO'] = muni
    map_rows.append(f)

map_df = pd.concat(map_rows, ignore_index=True)

# Filtrar filas sin coords
map_df = map_df.dropna(subset=['LONGITUD','LATITUD'])

max_global = map_df['VISITAS'].max()
map_df['INTENSIDAD_FORZADA'] = map_df['VISITAS'] / (max_global if max_global > 0 else 1)
map_df['INTENSIDAD_FORZADA'] = map_df['INTENSIDAD_FORZADA'].clip(lower=0.05)
map_df['MES_STR'] = map_df['ds'].dt.strftime('%Y-%m')
map_df['INTENSIDAD_MES_FORZADA'] = map_df.groupby('MES_STR')['VISITAS'].transform(
    lambda x: (x / (x.max() if x.max() > 0 else 1)).clip(lower=0.05)
)
map_df.to_csv(os.path.join(OUTPUT_DIR, 'map_df.csv'), index=False)

## Visualizacion con mapa

In [34]:
# Crear un mapa con HeatMapWithTime + MarkerCluster.
# HeatMapWithTime espera una lista de matrices de [lat,lon,intensity]
months = sorted(map_df['MES_STR'].unique())
heat_data = []
for m in months:
    sub = map_df[map_df['MES_STR'] == m]
    # HeatMap expects [lat, lon, weight]
    heat_data.append(sub[['LATITUD','LONGITUD','INTENSIDAD_FORZADA']].values.tolist())

# Calcular centro del mapa
mid_lat = map_df['LATITUD'].mean()
mid_lon = map_df['LONGITUD'].mean()
# Crear mapa base
m = folium.Map(location=[mid_lat, mid_lon], zoom_start=6)

# Porpiedades mapa de calor
HeatMapWithTime(
    heat_data,
    index=months,
    radius=25,
    auto_play=False,
    max_opacity=0.8
).add_to(m)

# Organizacion de marcadores por clusters
mc = MarkerCluster()
for _, row in map_df[map_df['ds'] == map_df['ds'].min()].iterrows():
    folium.Marker(
        location=[row['LATITUD'], row['LONGITUD']],
        popup=f"{row['MUNICIPIO']}"
    ).add_to(mc)
mc.add_to(m)

# Guardar mapa HTML
map_html_path = os.path.join(OUTPUT_DIR, 'Mapa_de_calor.html')
m.save(map_html_path)
print('Mapa guardado en:', map_html_path)


Mapa guardado en: Resultados prediccion Top 10\Mapa_de_calor.html
